# 과제
## 2022235027 민현기

1. 대구의 7개 구(중구, 동구, 서구, 남구, 북구, 수성구, 달서구)과 1개 군(달성군)의 맛집은 각각 몇 개인가?

2. 오픈 api에 포함된 "설명"(SMPL_DESC)을 분석하여, 맛집 설명에서 가장 유의미하게 많이 등장하는 5개 단어를 제시하라. 

3. 오픈 api에 포함된 "메뉴"(MNU)를 분석하여, 대구 맛집들이 제공하는 가장 흔한 메뉴 3개를 제시하라. 그리고 이들의 평균 가격 또한 함께 제시하라.

4. 본인이 제시한 2번, 3번 결과에 부합하는 맛집을 최종적으로 최대 3곳 선정하여 제시하라. (논리적 근거를 명확히 밝히라.)

 

* 답변의 제출은 반드시 "학번.pdf"로 제출하고, 추가적으로 분석에 활용된 python code를 원본형식으로 제출하라.

(코드 파일에 # 이나 Jupyternote book 기능을 이용해 설명만 추가하는 형식의 제출은 정식 과제 점수로 인정하지 않음.)

In [1]:
import requests
from urllib import parse 

In [2]:
def matzip_api(coordinate):
    address = 'https://www.daegufood.go.kr/kor/api/tasty.html?mode=json&addr='
    coordinate = parse.quote(coordinate)
    response = requests.get(address+coordinate)
    
    return response.json(strict=False)

In [3]:
location = ['중구', '동구', '서구', '남구', '북구', '수성구', '달서구', '달성군']

In [4]:
daegu_matzip = {i:matzip_api(i) for i in location}

### 1. 대구의 7개 구(중구, 동구, 서구, 남구, 북구, 수성구, 달서구)과 1개 군(달성군)의 맛집은 각각 몇 개인가?

In [5]:
for i in daegu_matzip:
    print(f'{i}: {daegu_matzip[i]["total"]} 개')

중구: 181 개
동구: 137 개
서구: 63 개
남구: 57 개
북구: 95 개
수성구: 131 개
달서구: 168 개
달성군: 93 개


### 2. 오픈 api에 포함된 "설명"(SMPL_DESC)을 분석하여, 맛집 설명에서 가장 유의미하게 많이 등장하는 5개 단어를 제시하라.

In [6]:
import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
import gensim
from gensim import corpora

In [7]:
def make_df(location):
    df = []
    for i in location:
        local_df = pd.DataFrame(daegu_matzip[i]['data'])
        local_df['LOCATION'] = i
        df.append(local_df)
        
    return pd.concat(df).reset_index(drop=True)

In [8]:
df = make_df(location)

In [9]:
df['FD_CS'].value_counts()

한식           700
양식            41
일식            40
디저트/베이커리      38
중식            36
세계요리          34
전통차/커피전문점     19
퓨전/뷔페         11
특별한 술집         6
Name: FD_CS, dtype: int64

In [17]:
class CustomLDA:
    def __init__(self, pandas_text_column, stop_words, num_topics=1, num_words=5):
        clean = re.compile("[^ㄱ-힣0-9 %]")
        self.texts = pandas_text_column.apply(lambda x: clean.sub('', str(x)))
        self.tagger = Okt()
        self.stop_words = stop_words
        self.num_topics = num_topics
        self.num_words = num_words
        
    def nouns_tokenizer(self, text):
        word_token = self.tagger.nouns(text)
        self.result = [word for word in word_token if word not in self.stop_words]
        return self.result
    
    def LDA(self):
        nouns_tokenizer = self.nouns_tokenizer
        texts = self.texts.apply(nouns_tokenizer)
        dictionary = corpora.Dictionary(texts)
        corpus = [dictionary.doc2bow(text) for text in texts]
        NUM_TOPICS = self.num_topics
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
        topics = ldamodel.print_topics(num_words=self.num_words)
        return topics

- 처음엔 불용어를 설정하지 않고, 결과를 돌려보며 리스트에 나오는 불용어를 하나씩 제거하며 최종적으로 5개의 단어를 선정함

In [11]:
stop_words = ['수', '요리', '곳', '등', '음식점', '한우', '사용', '음식']
CustomLDA(pandas_text_column=df['SMPL_DESC'], stop_words=stop_words, num_topics=1).LDA()

[(0, '0.043*"전문점" + 0.012*"맛" + 0.010*"위치" + 0.008*"전통" + 0.008*"직접"')]

### 3. 오픈 api에 포함된 "메뉴"(MNU)를 분석하여, 대구 맛집들이 제공하는 가장 흔한 메뉴 3개를 제시하라. 그리고 이들의 평균 가격 또한 함께 제시하라.

In [12]:
def make_menu_df(text):
    text = text.strip('\r\n').split('\r\n')
    
    pattern_price = re.compile('[0-9,[0-9]+(?=원)')
    pattern_menu = re.compile('^(.*?)(?= \d+,)')
    
    menu = pd.Series([pattern_menu.findall(i) for i in text]).apply(lambda x: ''.join(x) if x else '없음')
    # menu = pd.Series(text).apply(CustomLDA(pd.Series(text[0]), stop_words).nouns_tokenizer).apply(lambda x: ''.join(x))
    price = pd.Series([pattern_price.findall(i) for i in text]).apply(lambda x: int(x[0].replace(',', '')) if x else np.nan)
    
    return pd.DataFrame(zip(menu, price), columns=['품목', '가격'])

In [13]:
def menu_concat(df):
    concat_list_df = [make_menu_df(df['MNU'][i]) for i in range(len(df['MNU']))]
    return pd.concat(concat_list_df).reset_index(drop=True)

In [14]:
menu_df = menu_concat(df)

- 처음엔 불용어를 설정하지 않고, 결과를 돌려보며 리스트에 나오는 불용어를 하나씩 제거하며 최종적으로 각 토픽별 첫번째 메뉴를 대표메뉴로 선정함

In [19]:
stop_words_menu = ['세트', '인', '생']
CustomLDA(menu_df['품목'], stop_words=stop_words_menu, num_topics=3, num_words=3).LDA()

[(0, '0.043*"불고기" + 0.035*"정식" + 0.029*"탕"'),
 (1, '0.039*"찜" + 0.017*"삼계탕" + 0.016*"해물"'),
 (2, '0.032*"한우" + 0.028*"육회" + 0.024*"갈비살"')]

In [24]:
menu_df[menu_df['품목'].str.contains('불고기')]['가격'].mean()

17206.611570247933

In [25]:
menu_df[menu_df['품목'].str.contains('찜')]['가격'].mean()

30502.875399361023

In [26]:
menu_df[menu_df['품목'].str.contains('한우')]['가격'].mean()

22696.753246753247